# Generate Function Call Data for Travel Planning

In [ ]:
from langchain.tools import Tool# python
from pydantic import BaseModel, Field
from typing import Any

## Output Schema for LLM Prompts

In [125]:
class Row(BaseModel):
    question: str = Field(..., description="The input question")
    function_calls: list = Field(..., description="List of function calls with arguments")
    #available_functions: list = Field(..., description="List of available functions with name, description, parameters")

class Function(BaseModel):
    name: str = Field(..., description="The name of the function")
    description: str = Field(..., description="The description of the function")
    parameters: dict = Field(..., description="The parameters of the function in JSON Schema format")

class FunctionParameters(BaseModel):
    type: str = "object"
    properties: dict = Field(..., description="The properties of the parameters object")
    required: list = Field(..., description="The required fields of the parameters object")
    
class FunctionParam(BaseModel):
    type: str = Field(..., description="The type of the parameter")
    description: str = Field(..., description="The description of the parameter")
    default: Any = Field(None, description="The default value of the parameter, if any")
    minimum: int = Field(None, description="The minimum value for integer parameters, if any")
    enum: list = Field(None, description="The enumeration of possible values for the parameter"),
    
class FunctionCall(BaseModel):
    name: str = Field(..., description="The name of the function being called")
    arguments: dict = Field(..., description="List of arguments for the function call")
    

## Data Processing Utils

In [174]:
import random

def generate_function_call_data(rows, functions):
    function_data = []

    for doc in rows:
        question = doc.question
        
        function_call_list = [{"name":func_call.name, "arguments": func_call.arguments} for func_call in doc.function_calls]
        function_call_name_list = set([func_call["name"] for func_call in function_call_list])
        
        selected_functions = [func for func in functions if func["name"] in function_call_name_list]
        
        non_selected_functions = [func for func in functions if func["name"] not in function_call_name_list]
        n_functions_to_sample = len(functions)-len(selected_functions)-1 # leave one out for diversity
        available_functions = random.sample(non_selected_functions, n_functions_to_sample)+selected_functions
        
        # we want different orders
        random.shuffle(available_functions)
        
        available_functions = [{"name":func["name"], "description":func["description"], "parameters":func["parameters"] } for func in available_functions]
        
        function_data.append({
            "question":question,
            "function_calls":function_call_list,
            "available_functions":available_functions
            }
        )

    return function_data

In [179]:
import json
def save_json(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

## Functions Signatures

In [ ]:
def search_web(query:str):
    pass

def search_flight(fromCity, toCity, departureDate, returnDate, passengers:int):
    pass

def search_hotel(city, checkInDate, checkOutDate, guests:int):
    pass

def list_attractions(city:str, limit:int=10):
    pass

def search_youtube(query):
    pass

def list_events(city, startDate, endDate):
    pass

## Function Schemas

In [112]:
functions = [
{
  "name": "search_web",
  "description": "Search the web for the given query. Returns an array of result strings (title + URL).",
  "parameters": {
    "type": "object",
    "properties": {
      "query": { "type": "string", "description": "Search query text" }
    },
    "required": ["query"]
  }
},
 {
        "name": "search_flight",
        "description": "Search for round-trip flights for the given dates and number of passengers. Returns an array of flight objects (airline, flightNumber, departureDateTime, arrivalDateTime, duration, stops, price {amount, currency}, bookingUrl). Dates should be ISO 8601 (YYYY-MM-DD).",
        "parameters": {
            "type": "object",
            "properties": {
                "departureDate": {
                    "type": "string",
                    "description": "Outbound departure date in YYYY-MM-DD (ISO 8601).",
                    "format": "date"
                },
                "returnDate": {
                    "type": "string",
                    "description": "Return date in YYYY-MM-DD (ISO 8601).",
                    "format": "date"
                },
                "passengers": {
                    "type": "integer",
                    "description": "Number of passengers (integer >= 1).",
                    "minimum": 1
                }
            },
            "required": ["departureDate", "returnDate", "passengers"]
        }
    },
 
{
    "name": "search_hotel",
    "description": "Search for hotels for the given check-in/check-out dates and number of guests. Returns an array of hotel objects (name, address, rating, availableRooms, price {amount, currency}, bookingUrl). Dates should be ISO 8601 (YYYY-MM-DD).",
    "parameters": {
      "type": "object",
      "properties": {
        "checkInDate": {
          "type": "string",
          "description": "Check-in date in YYYY-MM-DD (ISO 8601).",
          "format": "date"
        },
        "checkOutDate": {
          "type": "string",
          "description": "Check-out date in YYYY-MM-DD (ISO 8601).",
          "format": "date"
        },
        "guests": {
          "type": "integer",
          "description": "Number of guests (integer >= 1).",
          "minimum": 1
        }
      },
      "required": ["checkInDate", "checkOutDate", "guests"]
    }
  },
  {
    "name": "list_attractions",
    "description": "List top attractions for the given city. Returns an array of attraction objects (name, address, shortDescription, rating, category).",
    "parameters": {
      "type": "object",
      "properties": {
        "city": {
          "type": "string",
          "description": "City name or query (e.g., 'Paris, France' or 'Seattle, WA')."
        },
        "limit": {
          "type": "integer",
          "description": "Maximum number of attractions to return.",
          "minimum": 1,
          "default": 10
        }
      },
      "required": ["city"]
    }
  }
,

{
    "name": "search_youtube",
    "description": "Search YouTube for videos related to the query and transcribe them to text. Primarily can be used to find travel blog videos and extract travel details in text. Returns an array of video objects: { title, url, transcript (string) }.",
    "parameters": {
      "type": "object",
      "properties": {
        "query": {
          "type": "string",
          "description": "Search query text (e.g., 'London in 5 days', 'weekend Paris tour')."
        }
      },
      "required": ["query"]
    }
  },

{
    "name": "list_events",
    "description": "List events in a city between startDate and endDate that may affect travel planning (e.g., strikes, bank holidays, festivals, carnivals). Returns an array of event objects: { id, name, type, description, startDateTime (ISO 8601), endDateTime (ISO 8601), allDay (boolean), location, timezone, impact (low|medium|high), isPublicHoliday (boolean), sourceUrl }.",
    "parameters": {
      "type": "object",
      "properties": {
        "city": {
          "type": "string",
          "description": "City name or query (e.g., 'Paris, France' or 'Seattle, WA')."
        },
        "startDate": {
          "type": "string",
          "description": "Start of the date range (inclusive) in YYYY-MM-DD (ISO 8601)."
        },
        "endDate": {
          "type": "string",
          "description": "End of the date range (inclusive) in YYYY-MM-DD (ISO 8601)."
        }
      },
      "required": ["city", "startDate", "endDate"]
    }
  }
]

In [194]:
save_json(functions, "functions.json")

## LLM Generated Function Call Data

### Web Search Data

In [141]:
# Rows with questions about traveling to East Asia and Australia/New Zealand
rows = [
    Row(
        question="İstanbul'da gezilecek tarihi yerler nerelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "İstanbul tarihi yerler listesi"}
            )
        ]
    ),
    Row(
        question="Kapadokya'da balon turları hakkında bilgi alabilir miyim?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Kapadokya balon turları fiyatları ve saatleri"}
            )
        ]
    ),
    Row(
        question="Antalya'da en iyi plajlar hangileridir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Antalya en iyi plajlar"}
            )
        ]
    ),
    Row(
        question="Pamukkale travertenlerini ziyaret etmek için en uygun zaman nedir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Pamukkale travertenleri ziyaret için en uygun zaman"}
            )
        ]
    ),
    Row(
        question="İzmir'de uygun fiyatlı oteller nerede bulunur?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "İzmir uygun fiyatlı oteller"}
            )
        ]
    ),
    Row(
        question="Bodrum'da gece hayatı nasıldır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Bodrum gece hayatı mekanları"}
            )
        ]
    ),
    Row(
        question="Efes Antik Kenti'ni ziyaret etmek için rehber hizmeti alabilir miyim?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Efes Antik Kenti rehber hizmeti"}
            )
        ]
    ),
    Row(
        question="Trabzon'da Sümela Manastırı'na nasıl gidilir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Trabzon Sümela Manastırı ulaşım"}
            )
        ]
    ),
    Row(
        question="Ankara'da Atatürk Orman Çiftliği'nde yapılacak aktiviteler nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Atatürk Orman Çiftliği aktiviteler"}
            )
        ]
    ),
    Row(
        question="Fethiye'de Ölüdeniz'de yapılacak su sporları nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Fethiye Ölüdeniz su sporları"}
            )
        ]
    )
    ,
     Row(
        question="Bugün İstanbul'da yapılacak etkinlikler nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "İstanbul günlük etkinlikler 26 Ağustos 2025"}
            )
        ]
    ),
    Row(
        question="Ankara'da bir günlük gezi için önerilen rotalar nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Ankara bir günlük gezi rotası"}
            )
        ]
    ),
    Row(
        question="İzmir'de bir gün içinde gezilecek en popüler yerler nerelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "İzmir bir günlük gezi popüler yerler"}
            )
        ]
    ),
    Row(
        question="Bugün Kapadokya'da hava durumu nasıl ve balon turları yapılacak mı?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Kapadokya hava durumu 26 Ağustos 2025 balon turları"}
            )
        ]
    ),
    Row(
        question="Antalya'da bir günlük tatil için en iyi plaj hangisidir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Antalya bir günlük tatil en iyi plaj"}
            )
        ]
    ),
    Row(
        question="Bugün Bodrum'da yapılacak kültürel etkinlikler nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Bodrum kültürel etkinlikler 26 Ağustos 2025"}
            )
        ]
    ),
    Row(
        question="Fethiye'de bir gün içinde yapılacak doğa aktiviteleri nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Fethiye bir günlük doğa aktiviteleri"}
            )
        ]
    ),
    Row(
        question="Bugün Trabzon'da Sümela Manastırı açık mı ve ziyaret saatleri nedir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Trabzon Sümela Manastırı açık mı 26 Ağustos 2025"}
            )
        ]
    ),
    Row(
        question="Pamukkale'de bir gün içinde yapılacak en iyi aktiviteler nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Pamukkale bir günlük en iyi aktiviteler"}
            )
        ]
    ),
    Row(
        question="Bugün Çanakkale'de Truva Antik Kenti'ni ziyaret etmek için bilgi alabilir miyim?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Çanakkale Truva Antik Kenti ziyaret bilgisi 26 Ağustos 2025"}
            )
        ]
    ),
     Row(
        question="Paris'te 3 günlük bir gezi için en iyi rota nedir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Paris 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Paris'te 3 günde gezilecek yerler"}
            )
        ]
    ),
    Row(
        question="Roma'da 4 günlük bir tatil için tarihi ve turistik yerler nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Roma 4 günlük tatil planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Roma'da 4 günde gezilecek tarihi yerler"}
            )
        ]
    ),
    Row(
        question="Amsterdam'da 2 günlük bir gezi için en iyi aktiviteler nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Amsterdam 2 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Amsterdam'da 2 günde yapılacak aktiviteler"}
            )
        ]
    ),
    Row(
        question="Barselona'da 5 günlük bir tatil için en iyi gezi rotası nedir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Barselona 5 günlük tatil planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Barselona'da 5 günde gezilecek yerler"}
            )
        ]
    ),
    Row(
        question="Prag'da 3 günlük bir gezi için tarihi yerler ve restoran önerileri nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Prag 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Prag'da 3 günde gezilecek tarihi yerler"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Prag restoran önerileri"}
            )
        ]
    ),
    Row(
        question="Viyana'da 2 günlük bir gezi için klasik müzik etkinlikleri ve saray ziyaretleri nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Viyana 2 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Viyana klasik müzik etkinlikleri"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Viyana saray ziyaret rehberi"}
            )
        ]
    ),
    Row(
        question="Berlin'de 4 günlük bir tatil için tarihi yerler ve müzeler nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Berlin 4 günlük tatil planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Berlin'de 4 günde gezilecek tarihi yerler"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Berlin müze ziyaret rehberi"}
            )
        ]
    ),
    Row(
        question="Londra'da 3 günlük bir gezi için en iyi turistik yerler ve alışveriş noktaları nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Londra 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Londra'da 3 günde gezilecek turistik yerler"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Londra alışveriş noktaları"}
            )
        ]
    ),
    Row(
        question="Floransa'da 2 günlük bir gezi için sanat galerileri ve tarihi yerler nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Floransa 2 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Floransa sanat galerileri ziyaret rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Floransa tarihi yerler gezi rehberi"}
            )
        ]
    ),
    Row(
        question="Budapeşte'de 3 günlük bir tatil için termal hamamlar ve tarihi yerler nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Budapeşte 3 günlük tatil planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Budapeşte termal hamamlar rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Budapeşte tarihi yerler gezi rehberi"}
            )
        ]
    )
    ,
     # North Africa
    Row(
        question="Marrakeş'te 3 günlük bir gezi için en iyi turistik yerler ve pazarlar nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Marrakeş 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Marrakeş'te gezilecek turistik yerler"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Marrakeş pazarlar ve alışveriş rehberi"}
            )
        ]
    ),
    Row(
        question="Kahire'de 2 günlük bir gezi için piramitler ve müzeler nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Kahire 2 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Kahire piramitler ziyaret rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Kahire müzeler gezi rehberi"}
            )
        ]
    ),
    Row(
        question="Tunus'ta 4 günlük bir tatil için tarihi yerler ve sahil aktiviteleri nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Tunus 4 günlük tatil planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Tunus tarihi yerler gezi rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Tunus sahil aktiviteleri"}
            )
        ]
    ),
    Row(
        question="Fas'ta 5 günlük bir gezi için Kazablanka ve Fes şehirleri nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Fas 5 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Kazablanka'da gezilecek yerler"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Fes'te gezilecek yerler"}
            )
        ]
    ),
    Row(
        question="Cezayir'de 3 günlük bir gezi için tarihi yerler ve çöl turları nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Cezayir 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Cezayir tarihi yerler gezi rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Cezayir çöl turları rehberi"}
            )
        ]
    ),
    
    # North America
    Row(
        question="New York'ta 3 günlük bir gezi için en iyi turistik yerler ve restoranlar nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "New York 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "New York'ta gezilecek turistik yerler"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "New York restoran önerileri"}
            )
        ]
    ),
    Row(
        question="Los Angeles'ta 4 günlük bir tatil için plajlar ve Hollywood turları nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Los Angeles 4 günlük tatil planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Los Angeles plajlar rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Los Angeles Hollywood turları"}
            )
        ]
    ),
    Row(
        question="Toronto'da 2 günlük bir gezi için CN Tower ve şehir merkezinde yapılacak aktiviteler nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Toronto 2 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Toronto CN Tower ziyaret rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Toronto şehir merkezi aktiviteleri"}
            )
        ]
    ),
    Row(
        question="Chicago'da 3 günlük bir gezi için müzeler ve göl kenarında yapılacak aktiviteler nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Chicago 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Chicago müzeler rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Chicago göl kenarı aktiviteleri"}
            )
        ]
    ),
    Row(
        question="Meksika'da 5 günlük bir tatil için Cancun ve Chichen Itza nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Meksika 5 günlük tatil planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Cancun'da yapılacak aktiviteler"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Chichen Itza ziyaret rehberi"}
            )
        ]
    ),
    # East Asia
    Row(
        question="Tokyo'da 3 günlük bir gezi için en iyi turistik yerler ve yemek mekanları nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Tokyo 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Tokyo'da gezilecek turistik yerler"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Tokyo yemek mekanları önerileri"}
            )
        ]
    ),
    Row(
        question="Seul'de 4 günlük bir tatil için tarihi yerler ve alışveriş noktaları nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Seul 4 günlük tatil planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Seul tarihi yerler rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Seul alışveriş noktaları"}
            )
        ]
    ),
    Row(
        question="Pekin'de 3 günlük bir gezi için Çin Seddi ve Yasak Şehir nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Pekin 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Çin Seddi ziyaret rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Pekin Yasak Şehir ziyaret rehberi"}
            )
        ]
    ),
    Row(
        question="Hong Kong'da 2 günlük bir gezi için en iyi aktiviteler ve manzaralı yerler nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Hong Kong 2 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Hong Kong'da yapılacak en iyi aktiviteler"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Hong Kong manzaralı yerler rehberi"}
            )
        ]
    ),
    Row(
        question="Kyoto'da 3 günlük bir gezi için tapınaklar ve geleneksel Japon bahçeleri nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Kyoto 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Kyoto tapınaklar rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Kyoto geleneksel Japon bahçeleri"}
            )
        ]
    ),
    
    # Australia and New Zealand
    Row(
        question="Sidney'de 3 günlük bir gezi için Opera Binası ve Bondi Plajı nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Sidney 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Sidney Opera Binası ziyaret rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Sidney Bondi Plajı aktiviteleri"}
            )
        ]
    ),
    Row(
        question="Melbourne'de 4 günlük bir tatil için Great Ocean Road ve şehir içi aktiviteler nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Melbourne 4 günlük tatil planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Great Ocean Road gezi rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Melbourne şehir içi aktiviteleri"}
            )
        ]
    ),
    Row(
        question="Auckland'da 3 günlük bir gezi için Waiheke Adası ve Sky Tower nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Auckland 3 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Auckland Waiheke Adası aktiviteleri"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Auckland Sky Tower ziyaret rehberi"}
            )
        ]
    ),
    Row(
        question="Queenstown'da 4 günlük bir tatil için macera sporları ve Milford Sound nasıl planlanır?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Queenstown 4 günlük tatil planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Queenstown macera sporları rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Milford Sound gezi rehberi"}
            )
        ]
    ),
    Row(
        question="Brisbane'de 2 günlük bir gezi için Lone Pine Koala Sanctuary ve South Bank aktiviteleri nelerdir?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Brisbane 2 günlük gezi planı"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Lone Pine Koala Sanctuary ziyaret rehberi"}
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Brisbane South Bank aktiviteleri"}
            )
        ]
    )
]

In [143]:
rows[0]

Row(question="İstanbul'da gezilecek tarihi yerler nerelerdir?", function_calls=[FunctionCall(name='search_web', arguments={'query': 'İstanbul tarihi yerler listesi'})])

In [ ]:
import random

function_name = "search_web"

current_function = [func for func in functions if func["name"] == function_name][0]
function_description = current_function["description"]
available_functions = [func for func in functions if func["name"] != function_name]


web_search_function_data = []

for doc in rows:
    question = doc.question
    
    function_call_list = [{"name":func_call.name, "arguments": func_call.arguments} for func_call in doc.function_calls]
    
    available_functions = random.sample(functions, 3)+[current_function]
    random.shuffle(available_functions)
    available_functions = [{"name":func["name"], "description":func["description"], "parameters":func["parameters"] } for func in available_functions]
    web_search_function_data.append({
        "question":question,
        "function_calls":function_call_list,
        "available_functions":available_functions
        }
    )

In [145]:
len(web_search_function_data)

50

In [176]:
web_search_function_calls = generate_function_call_data(rows, functions)

In [177]:
len(web_search_function_calls)

50

In [180]:
save_json(web_search_function_calls, "web_search_function_calls.json")

### Flight search data

In [153]:
flight_data = [
    Row(
        question="İstanbul'dan Paris'e 15 Eylül'de gidiş ve 20 Eylül'de dönüş için 2 kişilik uçuşları nasıl bulabilirim?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Paris",
                    "departureDate": "2025-09-15",
                    "returnDate": "2025-09-20",
                    "passengers": 2
                }
            )
        ]
    ),
    Row(
        question="Ankara'dan Londra'ya 1 Ekim'de gidiş ve 10 Ekim'de dönüş için 1 kişilik uçuşları nasıl bulabilirim?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Ankara",
                    "toCity": "Londra",
                    "departureDate": "2025-10-01",
                    "returnDate": "2025-10-10",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="New York'tan Tokyo'ya 5 Kasım'da gidiş ve 15 Kasım'da dönüş için 3 kişilik uçuşları nasıl bulabilirim?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "New York",
                    "toCity": "Tokyo",
                    "departureDate": "2025-11-05",
                    "returnDate": "2025-11-15",
                    "passengers": 3
                }
            )
        ]
    ),
    Row(
        question="Berlin'den Roma'ya 20 Aralık'ta gidiş ve 27 Aralık'ta dönüş için 4 kişilik uçuşları nasıl bulabilirim?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Berlin",
                    "toCity": "Roma",
                    "departureDate": "2025-12-20",
                    "returnDate": "2025-12-27",
                    "passengers": 4
                }
            )
        ]
    ),
    Row(
        question="Sydney'den Auckland'a 10 Ocak'ta gidiş ve 15 Ocak'ta dönüş için 2 kişilik uçuşları nasıl bulabilirim?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Sydney",
                    "toCity": "Auckland",
                    "departureDate": "2026-01-10",
                    "returnDate": "2026-01-15",
                    "passengers": 2
                }
            )
        ]
    ),
    Row(
        question="Toronto'dan Los Angeles'a 5 Şubat'ta gidiş ve 12 Şubat'ta dönüş için 1 kişilik uçuşları nasıl bulabilirim?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Toronto",
                    "toCity": "Los Angeles",
                    "departureDate": "2026-02-05",
                    "returnDate": "2026-02-12",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Cape Town'dan Nairobi'ye 15 Mart'ta gidiş ve 22 Mart'ta dönüş için 2 kişilik uçuşları nasıl bulabilirim?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Cape Town",
                    "toCity": "Nairobi",
                    "departureDate": "2026-03-15",
                    "returnDate": "2026-03-22",
                    "passengers": 2
                }
            )
        ]
    ),
    Row(
        question="Bangkok'tan Singapur'a 1 Nisan'da gidiş ve 5 Nisan'da dönüş için 3 kişilik uçuşları nasıl bulabilirim?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Bangkok",
                    "toCity": "Singapur",
                    "departureDate": "2026-04-01",
                    "returnDate": "2026-04-05",
                    "passengers": 3
                }
            )
        ]
    ),
    Row(
        question="Dubai'den Maldivler'e 10 Mayıs'ta gidiş ve 17 Mayıs'ta dönüş için 2 kişilik uçuşları nasıl bulabilirim?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Dubai",
                    "toCity": "Maldivler",
                    "departureDate": "2026-05-10",
                    "returnDate": "2026-05-17",
                    "passengers": 2
                }
            )
        ]
    ),
    Row(
        question="Moskova'dan İstanbul'a 20 Haziran'da gidiş ve 25 Haziran'da dönüş için 1 kişilik uçuşları nasıl bulabilirim?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Moskova",
                    "toCity": "İstanbul",
                    "departureDate": "2026-06-20",
                    "returnDate": "2026-06-25",
                    "passengers": 1
                }
            )
        ]
    )
    
    ,
    
    Row(
        question="Londra ekim ayı için 1 haftalık gezi planlıyorum. Uçuş planlaması yapar mısın?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Londra",
                    "departureDate": "2025-10-01",
                    "returnDate": "2025-10-08",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Eşimle umre yapmak istiyorum. En uygun fiyatlı gidiş dönüş uçuşlarını bulur musun?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Cidde",
                    "departureDate": "2025-12-01",
                    "returnDate": "2025-12-15",
                    "passengers": 2
                }
            )
        ]
    ),
    Row(
        question="New York'ta yılbaşı tatili için 1 haftalık bir gezi planlıyorum. Bana uygun uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "New York",
                    "departureDate": "2025-12-25",
                    "returnDate": "2026-01-01",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Ailemle birlikte 3 kişilik bir tatil için Tokyo'ya nisan ayında uçuş planlaması yapar mısın?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Tokyo",
                    "departureDate": "2026-04-10",
                    "returnDate": "2026-04-20",
                    "passengers": 3
                }
            )
        ]
    ),
    Row(
        question="Paris'te balayı için mayıs ayında 1 haftalık bir tatil planlıyoruz. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Paris",
                    "departureDate": "2026-05-15",
                    "returnDate": "2026-05-22",
                    "passengers": 2
                }
            )
        ]
    ),
    Row(
        question="Melbourne'de 10 günlük bir iş gezisi planlıyorum. Kasım ayı için uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Melbourne",
                    "departureDate": "2025-11-05",
                    "returnDate": "2025-11-15",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Dubai'de 5 günlük bir tatil yapmak istiyorum. Şubat ayında uygun uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Dubai",
                    "departureDate": "2026-02-10",
                    "returnDate": "2026-02-15",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Ailemle birlikte 4 kişilik bir tatil için Los Angeles'a yaz tatilinde uçuş planlaması yapar mısın?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Los Angeles",
                    "departureDate": "2026-07-01",
                    "returnDate": "2026-07-15",
                    "passengers": 4
                }
            )
        ]
    ),
    Row(
        question="Bangkok'ta 1 haftalık bir tatil için mart ayında uçuş planlaması yapabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Bangkok",
                    "departureDate": "2026-03-05",
                    "returnDate": "2026-03-12",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Berlin'de 3 günlük bir iş gezisi için ocak ayında uçuş planlaması yapar mısın?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Berlin",
                    "departureDate": "2026-01-10",
                    "returnDate": "2026-01-13",
                    "passengers": 1
                }
            )
        ]
    )
    
    
    ,
    
    
     Row(
        question="Londra'da 1 haftalık bir tatil planlıyorum. Gezilecek yerler ve uçuşlar hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Londra'da 1 haftada gezilecek yerler"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Londra",
                    "departureDate": "2025-10-01",
                    "returnDate": "2025-10-08",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Eşimle birlikte Paris'te balayı için 1 haftalık bir tatil planlıyoruz. Uçuşları ve romantik mekanları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Paris'te balayı için romantik mekanlar"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Paris",
                    "departureDate": "2026-05-15",
                    "returnDate": "2026-05-22",
                    "passengers": 2
                }
            )
        ]
    ),
    Row(
        question="New York'ta yılbaşı tatili için 1 haftalık bir gezi planlıyorum. Bana uçuşları ve yılbaşı etkinliklerini bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "New York yılbaşı etkinlikleri 2025"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "New York",
                    "departureDate": "2025-12-25",
                    "returnDate": "2026-01-01",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Tokyo'da 3 günlük bir gezi planlıyorum. Uçuşları ve gezilecek en iyi yerleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Tokyo'da 3 günde gezilecek en iyi yerler"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Tokyo",
                    "departureDate": "2026-04-10",
                    "returnDate": "2026-04-13",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Bangkok'ta 1 haftalık bir tatil için uçuşları ve en iyi sokak yemeklerini bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Bangkok'ta en iyi sokak yemekleri"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Bangkok",
                    "departureDate": "2026-03-05",
                    "returnDate": "2026-03-12",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Berlin'de 3 günlük bir iş gezisi için uçuşları ve akşam yemeği için restoran önerilerini bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Berlin'de iş gezisi için restoran önerileri"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Berlin",
                    "departureDate": "2026-01-10",
                    "returnDate": "2026-01-13",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Dubai'de 5 günlük bir tatil yapmak istiyorum. Uçuşları ve çöl safarisi turlarını bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Dubai çöl safarisi turları"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Dubai",
                    "departureDate": "2026-02-10",
                    "returnDate": "2026-02-15",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Los Angeles'ta yaz tatili için 2 haftalık bir gezi planlıyorum. Uçuşları ve plaj aktivitelerini bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Los Angeles'ta plaj aktiviteleri"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Los Angeles",
                    "departureDate": "2026-07-01",
                    "returnDate": "2026-07-15",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Melbourne'de 4 günlük bir iş gezisi planlıyorum. Uçuşları ve şehirde yapılacak akşam aktivitelerini bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Melbourne'de iş gezisi için akşam aktiviteleri"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Melbourne",
                    "departureDate": "2025-11-05",
                    "returnDate": "2025-11-09",
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Cape Town'da 1 haftalık bir tatil planlıyorum. Uçuşları ve safari turlarını bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Cape Town safari turları"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Cape Town",
                    "departureDate": "2026-03-15",
                    "returnDate": "2026-03-22",
                    "passengers": 1
                }
            )
        ]
    )
    
    ,
    
    Row(
        question="Avrupa turu planlıyorum. İstanbul'dan Paris'e gidip, oradan Londra'ya geçip, tekrar İstanbul'a dönmek istiyorum. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Paris",
                    "departureDate": "2025-09-15",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Paris",
                    "toCity": "Londra",
                    "departureDate": "2025-09-20",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Londra",
                    "toCity": "İstanbul",
                    "departureDate": "2025-09-25",
                    "returnDate": None,
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Asya turu planlıyorum. Tokyo'dan Seul'e, ardından Pekin'e ve son olarak Tokyo'ya dönmek istiyorum. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Tokyo",
                    "toCity": "Seul",
                    "departureDate": "2025-11-01",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Seul",
                    "toCity": "Pekin",
                    "departureDate": "2025-11-05",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Pekin",
                    "toCity": "Tokyo",
                    "departureDate": "2025-11-10",
                    "returnDate": None,
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Amerika turu planlıyorum. New York'tan Los Angeles'a, oradan Miami'ye ve tekrar New York'a dönmek istiyorum. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "New York",
                    "toCity": "Los Angeles",
                    "departureDate": "2025-12-01",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Los Angeles",
                    "toCity": "Miami",
                    "departureDate": "2025-12-05",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Miami",
                    "toCity": "New York",
                    "departureDate": "2025-12-10",
                    "returnDate": None,
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Avustralya ve Yeni Zelanda turu planlıyorum. Sidney'den Auckland'a, oradan Wellington'a ve tekrar Sidney'e dönmek istiyorum. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Sidney",
                    "toCity": "Auckland",
                    "departureDate": "2026-01-10",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Auckland",
                    "toCity": "Wellington",
                    "departureDate": "2026-01-15",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Wellington",
                    "toCity": "Sidney",
                    "departureDate": "2026-01-20",
                    "returnDate": None,
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Afrika turu planlıyorum. Kahire'den Nairobi'ye, oradan Cape Town'a ve tekrar Kahire'ye dönmek istiyorum. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Kahire",
                    "toCity": "Nairobi",
                    "departureDate": "2026-02-01",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Nairobi",
                    "toCity": "Cape Town",
                    "departureDate": "2026-02-07",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Cape Town",
                    "toCity": "Kahire",
                    "departureDate": "2026-02-14",
                    "returnDate": None,
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Güney Amerika turu planlıyorum. Buenos Aires'ten Rio de Janeiro'ya, oradan Lima'ya ve tekrar Buenos Aires'e dönmek istiyorum. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Buenos Aires",
                    "toCity": "Rio de Janeiro",
                    "departureDate": "2026-03-01",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Rio de Janeiro",
                    "toCity": "Lima",
                    "departureDate": "2026-03-06",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Lima",
                    "toCity": "Buenos Aires",
                    "departureDate": "2026-03-12",
                    "returnDate": None,
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Orta Doğu turu planlıyorum. İstanbul'dan Dubai'ye, oradan Doha'ya ve tekrar İstanbul'a dönmek istiyorum. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Dubai",
                    "departureDate": "2026-04-01",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Dubai",
                    "toCity": "Doha",
                    "departureDate": "2026-04-05",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Doha",
                    "toCity": "İstanbul",
                    "departureDate": "2026-04-10",
                    "returnDate": None,
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Karayipler turu planlıyorum. Miami'den Havana'ya, oradan Kingston'a ve tekrar Miami'ye dönmek istiyorum. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Miami",
                    "toCity": "Havana",
                    "departureDate": "2026-05-01",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Havana",
                    "toCity": "Kingston",
                    "departureDate": "2026-05-06",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Kingston",
                    "toCity": "Miami",
                    "departureDate": "2026-05-12",
                    "returnDate": None,
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Hint Okyanusu turu planlıyorum. Maldivler'den Seyşeller'e, oradan Mauritius'a ve tekrar Maldivler'e dönmek istiyorum. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Maldivler",
                    "toCity": "Seyşeller",
                    "departureDate": "2026-06-01",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Seyşeller",
                    "toCity": "Mauritius",
                    "departureDate": "2026-06-07",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Mauritius",
                    "toCity": "Maldivler",
                    "departureDate": "2026-06-14",
                    "returnDate": None,
                    "passengers": 1
                }
            )
        ]
    ),
    Row(
        question="Kuzey Avrupa turu planlıyorum. Stockholm'den Kopenhag'a, oradan Oslo'ya ve tekrar Stockholm'e dönmek istiyorum. Uçuşları bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Stockholm",
                    "toCity": "Kopenhag",
                    "departureDate": "2026-07-01",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Kopenhag",
                    "toCity": "Oslo",
                    "departureDate": "2026-07-05",
                    "returnDate": None,
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "Oslo",
                    "toCity": "Stockholm",
                    "departureDate": "2026-07-10",
                    "returnDate": None,
                    "passengers": 1
                }
            )
        ]
    ),
    
]

In [154]:
len(flight_data)

40

In [181]:
flight_search_function_calls = generate_function_call_data(flight_data, functions)

In [182]:
len(flight_search_function_calls)

40

In [183]:
save_json(web_search_function_calls, "flight_search_function_calls.json")

### Hotel Search Data

In [189]:
# Rows with questions for the search_hotel function
hote_search_data = [
    Row(
        question="Paris'te 15 Eylül'den 20 Eylül'e kadar 2 kişilik bir otel arıyorum. Bana uygun otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Paris",
                    "checkInDate": "2025-09-15",
                    "checkOutDate": "2025-09-20",
                    "guests": 2
                }
            )
        ]
    ),
    Row(
        question="Londra'da 1 Ekim'den 5 Ekim'e kadar 1 kişilik bir otel arıyorum. Yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Londra",
                    "checkInDate": "2025-10-01",
                    "checkOutDate": "2025-10-05",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="New York'ta yılbaşı tatili için 25 Aralık'tan 1 Ocak'a kadar 3 kişilik bir otel arıyorum. Otel önerisi yapabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "New York",
                    "checkInDate": "2025-12-25",
                    "checkOutDate": "2026-01-01",
                    "guests": 3
                }
            )
        ]
    ),
    Row(
        question="Tokyo'da 10 Nisan'dan 15 Nisan'a kadar 2 kişilik bir otel arıyorum. Bana uygun otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Tokyo",
                    "checkInDate": "2026-04-10",
                    "checkOutDate": "2026-04-15",
                    "guests": 2
                }
            )
        ]
    ),
    Row(
        question="Bangkok'ta 5 Mart'tan 12 Mart'a kadar 1 kişilik bir otel arıyorum. Yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Bangkok",
                    "checkInDate": "2026-03-05",
                    "checkOutDate": "2026-03-12",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Berlin'de 10 Ocak'tan 13 Ocak'a kadar 1 kişilik bir otel arıyorum. Bana uygun otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Berlin",
                    "checkInDate": "2026-01-10",
                    "checkOutDate": "2026-01-13",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Dubai'de 10 Şubat'tan 15 Şubat'a kadar 2 kişilik bir otel arıyorum. Otel önerisi yapabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Dubai",
                    "checkInDate": "2026-02-10",
                    "checkOutDate": "2026-02-15",
                    "guests": 2
                }
            )
        ]
    ),
    Row(
        question="Los Angeles'ta yaz tatili için 1 Temmuz'dan 15 Temmuz'a kadar 4 kişilik bir otel arıyorum. Yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Los Angeles",
                    "checkInDate": "2026-07-01",
                    "checkOutDate": "2026-07-15",
                    "guests": 4
                }
            )
        ]
    ),
    Row(
        question="Cape Town'da 15 Mart'tan 22 Mart'a kadar 1 kişilik bir otel arıyorum. Bana uygun otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Cape Town",
                    "checkInDate": "2026-03-15",
                    "checkOutDate": "2026-03-22",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Melbourne'de 5 Kasım'dan 10 Kasım'a kadar 1 kişilik bir otel arıyorum. Yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Melbourne",
                    "checkInDate": "2025-11-05",
                    "checkOutDate": "2025-11-10",
                    "guests": 1
                }
            )
        ]
    )
    
    ,
    
    Row(
        question="Avrupa turu planlıyorum. Paris, Londra ve Berlin'de konaklama için oteller arıyorum. Bana yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Paris",
                    "checkInDate": "2025-09-15",
                    "checkOutDate": "2025-09-18",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Londra",
                    "checkInDate": "2025-09-18",
                    "checkOutDate": "2025-09-21",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Berlin",
                    "checkInDate": "2025-09-21",
                    "checkOutDate": "2025-09-24",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Asya turu planlıyorum. Tokyo, Seul ve Pekin'de konaklama için oteller arıyorum. Yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Tokyo",
                    "checkInDate": "2025-11-01",
                    "checkOutDate": "2025-11-04",
                    "guests": 2
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Seul",
                    "checkInDate": "2025-11-04",
                    "checkOutDate": "2025-11-07",
                    "guests": 2
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Pekin",
                    "checkInDate": "2025-11-07",
                    "checkOutDate": "2025-11-10",
                    "guests": 2
                }
            )
        ]
    ),
    Row(
        question="Amerika turu planlıyorum. New York, Los Angeles ve Miami'de konaklama için oteller arıyorum. Bana yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "New York",
                    "checkInDate": "2025-12-01",
                    "checkOutDate": "2025-12-05",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Los Angeles",
                    "checkInDate": "2025-12-05",
                    "checkOutDate": "2025-12-09",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Miami",
                    "checkInDate": "2025-12-09",
                    "checkOutDate": "2025-12-13",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Avustralya ve Yeni Zelanda turu planlıyorum. Sidney, Auckland ve Wellington'da konaklama için oteller arıyorum. Yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Sidney",
                    "checkInDate": "2026-01-10",
                    "checkOutDate": "2026-01-13",
                    "guests": 2
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Auckland",
                    "checkInDate": "2026-01-13",
                    "checkOutDate": "2026-01-16",
                    "guests": 2
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Wellington",
                    "checkInDate": "2026-01-16",
                    "checkOutDate": "2026-01-19",
                    "guests": 2
                }
            )
        ]
    ),
    Row(
        question="Afrika turu planlıyorum. Kahire, Nairobi ve Cape Town'da konaklama için oteller arıyorum. Bana yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Kahire",
                    "checkInDate": "2026-02-01",
                    "checkOutDate": "2026-02-04",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Nairobi",
                    "checkInDate": "2026-02-04",
                    "checkOutDate": "2026-02-07",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Cape Town",
                    "checkInDate": "2026-02-07",
                    "checkOutDate": "2026-02-10",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Güney Amerika turu planlıyorum. Buenos Aires, Rio de Janeiro ve Lima'da konaklama için oteller arıyorum. Yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Buenos Aires",
                    "checkInDate": "2026-03-01",
                    "checkOutDate": "2026-03-04",
                    "guests": 2
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Rio de Janeiro",
                    "checkInDate": "2026-03-04",
                    "checkOutDate": "2026-03-07",
                    "guests": 2
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Lima",
                    "checkInDate": "2026-03-07",
                    "checkOutDate": "2026-03-10",
                    "guests": 2
                }
            )
        ]
    ),
    Row(
        question="Orta Doğu turu planlıyorum. İstanbul, Dubai ve Doha'da konaklama için oteller arıyorum. Bana yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "İstanbul",
                    "checkInDate": "2026-04-01",
                    "checkOutDate": "2026-04-04",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Dubai",
                    "checkInDate": "2026-04-04",
                    "checkOutDate": "2026-04-07",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Doha",
                    "checkInDate": "2026-04-07",
                    "checkOutDate": "2026-04-10",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Karayipler turu planlıyorum. Miami, Havana ve Kingston'da konaklama için oteller arıyorum. Yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Miami",
                    "checkInDate": "2026-05-01",
                    "checkOutDate": "2026-05-04",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Havana",
                    "checkInDate": "2026-05-04",
                    "checkOutDate": "2026-05-07",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Kingston",
                    "checkInDate": "2026-05-07",
                    "checkOutDate": "2026-05-10",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Hint Okyanusu turu planlıyorum. Maldivler, Seyşeller ve Mauritius'ta konaklama için oteller arıyorum. Bana yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Maldivler",
                    "checkInDate": "2026-06-01",
                    "checkOutDate": "2026-06-04",
                    "guests": 2
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Seyşeller",
                    "checkInDate": "2026-06-04",
                    "checkOutDate": "2026-06-07",
                    "guests": 2
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Mauritius",
                    "checkInDate": "2026-06-07",
                    "checkOutDate": "2026-06-10",
                    "guests": 2
                }
            )
        ]
    ),
    Row(
        question="Kuzey Avrupa turu planlıyorum. Stockholm, Kopenhag ve Oslo'da konaklama için oteller arıyorum. Yardımcı olabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Stockholm",
                    "checkInDate": "2026-07-01",
                    "checkOutDate": "2026-07-04",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Kopenhag",
                    "checkInDate": "2026-07-04",
                    "checkOutDate": "2026-07-07",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Oslo",
                    "checkInDate": "2026-07-07",
                    "checkOutDate": "2026-07-10",
                    "guests": 1
                }
            )
        ]
    )
    
    ,
    
    Row(
        question="Paris'te 15 Eylül'den 20 Eylül'e kadar bir tatil planlıyorum. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Paris",
                    "departureDate": "2025-09-15",
                    "returnDate": "2025-09-20",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Paris",
                    "checkInDate": "2025-09-15",
                    "checkOutDate": "2025-09-20",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Londra'da 1 Ekim'den 5 Ekim'e kadar bir iş gezisi planlıyorum. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Londra",
                    "departureDate": "2025-10-01",
                    "returnDate": "2025-10-05",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Londra",
                    "checkInDate": "2025-10-01",
                    "checkOutDate": "2025-10-05",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="New York'ta yılbaşı tatili için 25 Aralık'tan 1 Ocak'a kadar bir gezi planlıyorum. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "New York",
                    "departureDate": "2025-12-25",
                    "returnDate": "2026-01-01",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "New York",
                    "checkInDate": "2025-12-25",
                    "checkOutDate": "2026-01-01",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Tokyo'da 10 Nisan'dan 15 Nisan'a kadar bir tatil planlıyorum. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Tokyo",
                    "departureDate": "2026-04-10",
                    "returnDate": "2026-04-15",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Tokyo",
                    "checkInDate": "2026-04-10",
                    "checkOutDate": "2026-04-15",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Bangkok'ta 5 Mart'tan 12 Mart'a kadar bir tatil planlıyorum. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Bangkok",
                    "departureDate": "2026-03-05",
                    "returnDate": "2026-03-12",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Bangkok",
                    "checkInDate": "2026-03-05",
                    "checkOutDate": "2026-03-12",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Berlin'de 10 Ocak'tan 13 Ocak'a kadar bir iş gezisi planlıyorum. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Berlin",
                    "departureDate": "2026-01-10",
                    "returnDate": "2026-01-13",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Berlin",
                    "checkInDate": "2026-01-10",
                    "checkOutDate": "2026-01-13",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Dubai'de 10 Şubat'tan 15 Şubat'a kadar bir tatil planlıyorum. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Dubai",
                    "departureDate": "2026-02-10",
                    "returnDate": "2026-02-15",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Dubai",
                    "checkInDate": "2026-02-10",
                    "checkOutDate": "2026-02-15",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Los Angeles'ta yaz tatili için 1 Temmuz'dan 15 Temmuz'a kadar bir tatil planlıyorum. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Los Angeles",
                    "departureDate": "2026-07-01",
                    "returnDate": "2026-07-15",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Los Angeles",
                    "checkInDate": "2026-07-01",
                    "checkOutDate": "2026-07-15",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Cape Town'da 15 Mart'tan 22 Mart'a kadar bir tatil planlıyorum. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Cape Town",
                    "departureDate": "2026-03-15",
                    "returnDate": "2026-03-22",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Cape Town",
                    "checkInDate": "2026-03-15",
                    "checkOutDate": "2026-03-22",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Melbourne'de 5 Kasım'dan 10 Kasım'a kadar bir iş gezisi planlıyorum. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Melbourne",
                    "departureDate": "2025-11-05",
                    "returnDate": "2025-11-10",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Melbourne",
                    "checkInDate": "2025-11-05",
                    "checkOutDate": "2025-11-10",
                    "guests": 1
                }
            )
        ]
    )
    
    ,
    
    Row(
        question="Paris'te 15 Eylül'den 20 Eylül'e kadar bir tatil planlıyorum. Gezilecek yerler, uçuşlar ve oteller hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Paris'te 5 günde gezilecek yerler"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Paris",
                    "departureDate": "2025-09-15",
                    "returnDate": "2025-09-20",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Paris",
                    "checkInDate": "2025-09-15",
                    "checkOutDate": "2025-09-20",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Londra'da 1 Ekim'den 5 Ekim'e kadar bir iş gezisi planlıyorum. İş toplantıları için restoran önerileri, uçuşlar ve oteller hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Londra'da iş toplantıları için restoran önerileri"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Londra",
                    "departureDate": "2025-10-01",
                    "returnDate": "2025-10-05",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Londra",
                    "checkInDate": "2025-10-01",
                    "checkOutDate": "2025-10-05",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="New York'ta yılbaşı tatili için 25 Aralık'tan 1 Ocak'a kadar bir gezi planlıyorum. Yılbaşı etkinlikleri, uçuşlar ve oteller hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "New York yılbaşı etkinlikleri 2025"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "New York",
                    "departureDate": "2025-12-25",
                    "returnDate": "2026-01-01",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "New York",
                    "checkInDate": "2025-12-25",
                    "checkOutDate": "2026-01-01",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Tokyo'da 10 Nisan'dan 15 Nisan'a kadar bir tatil planlıyorum. Japon mutfağı restoran önerileri, uçuşlar ve oteller hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Tokyo'da Japon mutfağı restoran önerileri"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Tokyo",
                    "departureDate": "2026-04-10",
                    "returnDate": "2026-04-15",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Tokyo",
                    "checkInDate": "2026-04-10",
                    "checkOutDate": "2026-04-15",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Bangkok'ta 5 Mart'tan 12 Mart'a kadar bir tatil planlıyorum. Sokak yemekleri turları, uçuşlar ve oteller hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Bangkok'ta sokak yemekleri turları"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Bangkok",
                    "departureDate": "2026-03-05",
                    "returnDate": "2026-03-12",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Bangkok",
                    "checkInDate": "2026-03-05",
                    "checkOutDate": "2026-03-12",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Berlin'de 10 Ocak'tan 13 Ocak'a kadar bir iş gezisi planlıyorum. Akşam aktiviteleri, uçuşlar ve oteller hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Berlin'de iş gezisi için akşam aktiviteleri"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Berlin",
                    "departureDate": "2026-01-10",
                    "returnDate": "2026-01-13",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Berlin",
                    "checkInDate": "2026-01-10",
                    "checkOutDate": "2026-01-13",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Dubai'de 10 Şubat'tan 15 Şubat'a kadar bir tatil planlıyorum. Çöl safarisi turları, uçuşlar ve oteller hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Dubai çöl safarisi turları"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Dubai",
                    "departureDate": "2026-02-10",
                    "returnDate": "2026-02-15",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Dubai",
                    "checkInDate": "2026-02-10",
                    "checkOutDate": "2026-02-15",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Los Angeles'ta yaz tatili için 1 Temmuz'dan 15 Temmuz'a kadar bir tatil planlıyorum. Plaj aktiviteleri, uçuşlar ve oteller hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Los Angeles'ta plaj aktiviteleri"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Los Angeles",
                    "departureDate": "2026-07-01",
                    "returnDate": "2026-07-15",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Los Angeles",
                    "checkInDate": "2026-07-01",
                    "checkOutDate": "2026-07-15",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Cape Town'da 15 Mart'tan 22 Mart'a kadar bir tatil planlıyorum. Safari turları, uçuşlar ve oteller hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Cape Town safari turları"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Cape Town",
                    "departureDate": "2026-03-15",
                    "returnDate": "2026-03-22",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Cape Town",
                    "checkInDate": "2026-03-15",
                    "checkOutDate": "2026-03-22",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Melbourne'de 5 Kasım'dan 10 Kasım'a kadar bir iş gezisi planlıyorum. Şehirde yapılacak akşam aktiviteleri, uçuşlar ve oteller hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_web",
                arguments={"query": "Melbourne'de iş gezisi için akşam aktiviteleri"}
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Melbourne",
                    "departureDate": "2025-11-05",
                    "returnDate": "2025-11-10",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Melbourne",
                    "checkInDate": "2025-11-05",
                    "checkOutDate": "2025-11-10",
                    "guests": 1
                }
            )
        ]
    )
    
    ,
    
     Row(
        question="Paris'e 15 Eylül'de gitmek istiyorum ama uçuşlar pahalıysa alternatif olarak Amsterdam'a gitmeyi düşünebilirim. Uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Paris",
                    "departureDate": "2025-09-15",
                    "returnDate": "2025-09-20",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Amsterdam",
                    "departureDate": "2025-09-15",
                    "returnDate": "2025-09-20",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Paris",
                    "checkInDate": "2025-09-15",
                    "checkOutDate": "2025-09-20",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Amsterdam",
                    "checkInDate": "2025-09-15",
                    "checkOutDate": "2025-09-20",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="New York'ta yılbaşı tatili planlıyorum ama uçuşlar çok pahalıysa alternatif olarak Chicago'yu düşünebilirim. Bana her iki şehir için uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "New York",
                    "departureDate": "2025-12-25",
                    "returnDate": "2026-01-01",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Chicago",
                    "departureDate": "2025-12-25",
                    "returnDate": "2026-01-01",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "New York",
                    "checkInDate": "2025-12-25",
                    "checkOutDate": "2026-01-01",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Chicago",
                    "checkInDate": "2025-12-25",
                    "checkOutDate": "2026-01-01",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Tokyo'ya 10 Nisan'da gitmek istiyorum ama uçuşlar pahalıysa alternatif olarak Seul'e gitmeyi düşünebilirim. Her iki şehir için uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Tokyo",
                    "departureDate": "2026-04-10",
                    "returnDate": "2026-04-15",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Seul",
                    "departureDate": "2026-04-10",
                    "returnDate": "2026-04-15",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Tokyo",
                    "checkInDate": "2026-04-10",
                    "checkOutDate": "2026-04-15",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Seul",
                    "checkInDate": "2026-04-10",
                    "checkOutDate": "2026-04-15",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Bangkok'ta 5 Mart'tan 12 Mart'a kadar bir tatil planlıyorum ama uçuşlar pahalıysa alternatif olarak Kuala Lumpur'u düşünebilirim. Her iki şehir için uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Bangkok",
                    "departureDate": "2026-03-05",
                    "returnDate": "2026-03-12",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Kuala Lumpur",
                    "departureDate": "2026-03-05",
                    "returnDate": "2026-03-12",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Bangkok",
                    "checkInDate": "2026-03-05",
                    "checkOutDate": "2026-03-12",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Kuala Lumpur",
                    "checkInDate": "2026-03-05",
                    "checkOutDate": "2026-03-12",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Berlin'de 10 Ocak'tan 13 Ocak'a kadar bir iş gezisi planlıyorum ama oteller doluysa alternatif olarak Münih'i düşünebilirim. Her iki şehir için uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Berlin",
                    "departureDate": "2026-01-10",
                    "returnDate": "2026-01-13",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Münih",
                    "departureDate": "2026-01-10",
                    "returnDate": "2026-01-13",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Berlin",
                    "checkInDate": "2026-01-10",
                    "checkOutDate": "2026-01-13",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Münih",
                    "checkInDate": "2026-01-10",
                    "checkOutDate": "2026-01-13",
                    "guests": 1
                }
            )
        ]
    ),
    Row(
        question="Dubai'de 10 Şubat'tan 15 Şubat'a kadar bir tatil planlıyorum ama uçuşlar pahalıysa alternatif olarak Abu Dhabi'yi düşünebilirim. Her iki şehir için uçuşları ve otelleri bulabilir misin?",
        function_calls=[
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Dubai",
                    "departureDate": "2026-02-10",
                    "returnDate": "2026-02-15",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_flight",
                arguments={
                    "fromCity": "İstanbul",
                    "toCity": "Abu Dhabi",
                    "departureDate": "2026-02-10",
                    "returnDate": "2026-02-15",
                    "passengers": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Dubai",
                    "checkInDate": "2026-02-10",
                    "checkOutDate": "2026-02-15",
                    "guests": 1
                }
            ),
            FunctionCall(
                name="search_hotel",
                arguments={
                    "city": "Abu Dhabi",
                    "checkInDate": "2026-02-10",
                    "checkOutDate": "2026-02-15",
                    "guests": 1
                }
            )
        ]
    )
    
]

In [190]:
len(hote_search_data)

46

In [191]:
hote_search_function_calls = generate_function_call_data(hote_search_data, functions)

In [193]:
save_json(hote_search_function_calls, "hotel_search_function_calls.json")

### Attraction List Data

In [195]:
# Rows with questions for the list_attractions function
attraction_data = [
    Row(
        question="Paris'te gezilecek en popüler 10 yeri listeleyebilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Paris",
                    "limit": 10
                }
            )
        ]
    ),
    Row(
        question="Londra'da gezilecek en iyi 5 yeri öğrenebilir miyim?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Londra",
                    "limit": 5
                }
            )
        ]
    ),
    Row(
        question="New York'ta gezilecek en popüler yerleri listeleyebilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "New York",
                    "limit": 10
                }
            )
        ]
    ),
    Row(
        question="Tokyo'da gezilecek en iyi 7 yeri öğrenebilir miyim?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Tokyo",
                    "limit": 7
                }
            )
        ]
    ),
    Row(
        question="Bangkok'ta gezilecek en popüler yerleri listeleyebilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Bangkok",
                    "limit": 10
                }
            )
        ]
    ),
    Row(
        question="Berlin'de gezilecek en iyi 3 yeri öğrenebilir miyim?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Berlin",
                    "limit": 3
                }
            )
        ]
    ),
    Row(
        question="Dubai'de gezilecek en popüler 8 yeri listeleyebilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Dubai",
                    "limit": 8
                }
            )
        ]
    ),
    Row(
        question="Los Angeles'ta gezilecek en iyi 6 yeri öğrenebilir miyim?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Los Angeles",
                    "limit": 6
                }
            )
        ]
    ),
    Row(
        question="Cape Town'da gezilecek en popüler yerleri listeleyebilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Cape Town",
                    "limit": 10
                }
            )
        ]
    ),
    Row(
        question="Melbourne'de gezilecek en iyi 4 yeri öğrenebilir miyim?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Melbourne",
                    "limit": 4
                }
            )
        ]
    )
    ,
    
    Row(
        question="Avrupa turu planlıyorum. Paris, Londra ve Berlin'de gezilecek en popüler yerleri listeleyebilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Paris",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Londra",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Berlin",
                    "limit": 10
                }
            )
        ]
    ),
    Row(
        question="Asya turu planlıyorum. Tokyo, Bangkok ve Seul'de gezilecek en iyi 5 yeri öğrenebilir miyim?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Tokyo",
                    "limit": 5
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Bangkok",
                    "limit": 5
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Seul",
                    "limit": 5
                }
            )
        ]
    ),
    Row(
        question="Amerika turu planlıyorum. New York, Los Angeles ve Miami'de gezilecek en popüler yerleri listeleyebilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "New York",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Los Angeles",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Miami",
                    "limit": 10
                }
            )
        ]
    ),
    Row(
        question="Avustralya ve Yeni Zelanda turu planlıyorum. Sidney, Melbourne ve Auckland'da gezilecek en iyi 7 yeri öğrenebilir miyim?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Sidney",
                    "limit": 7
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Melbourne",
                    "limit": 7
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Auckland",
                    "limit": 7
                }
            )
        ]
    ),
    Row(
        question="Afrika turu planlıyorum. Kahire, Nairobi ve Cape Town'da gezilecek en popüler yerleri listeleyebilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Kahire",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Nairobi",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Cape Town",
                    "limit": 10
                }
            )
        ]
    ),
    Row(
        question="Güney Amerika turu planlıyorum. Buenos Aires, Rio de Janeiro ve Lima'da gezilecek en iyi 6 yeri öğrenebilir miyim?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Buenos Aires",
                    "limit": 6
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Rio de Janeiro",
                    "limit": 6
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Lima",
                    "limit": 6
                }
            )
        ]
    ),
    Row(
        question="Orta Doğu turu planlıyorum. İstanbul, Dubai ve Doha'da gezilecek en popüler yerleri listeleyebilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "İstanbul"
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Dubai"
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Doha"
                }
            )
        ]
    ),
    Row(
        question="Karayipler turu planlıyorum. Havana, Kingston ve Nassau'da gezilecek en iyi 5 yeri öğrenebilir miyim?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Havana",
                    "limit": 5
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Kingston",
                    "limit": 5
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Nassau",
                    "limit": 5
                }
            )
        ]
    ),
    Row(
        question="Hint Okyanusu turu planlıyorum. Maldivler, Seyşeller ve Mauritius'ta gezilecek en popüler yerleri listeleyebilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Maldivler"
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Seyşeller"
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Mauritius"
                }
            )
        ]
    ),
    Row(
        question="Kuzey Avrupa turu planlıyorum. Stockholm, Kopenhag ve Oslo'da gezilecek en iyi 8 yeri öğrenebilir miyim?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Stockholm",
                    "limit": 8
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Kopenhag",
                    "limit": 8
                }
            ),
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Oslo",
                    "limit": 8
                }
            )
        ]
    )
    ,
    
    Row(
        question="Paris'te 5 günlük bir tatil planlıyorum. Gezilecek yerler ve şehirdeki etkinlikler hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Paris",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Paris'te 5 günlük tatil için etkinlikler"}
            )
        ]
    ),
    Row(
        question="Londra'da 3 günlük bir iş gezisi planlıyorum. Gezilecek yerler ve iş toplantıları için restoran önerileri hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Londra",
                    "limit": 5
                }
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Londra'da iş toplantıları için restoran önerileri"}
            )
        ]
    ),
    Row(
        question="New York'ta yılbaşı tatili için gezilecek yerler ve yılbaşı etkinlikleri hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "New York",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "New York yılbaşı etkinlikleri 2025"}
            )
        ]
    ),
    Row(
        question="Tokyo'da 7 günlük bir tatil planlıyorum. Gezilecek yerler ve Japon mutfağı restoran önerileri hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Tokyo",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Tokyo'da Japon mutfağı restoran önerileri"}
            )
        ]
    ),
    Row(
        question="Bangkok'ta 1 haftalık bir tatil planlıyorum. Gezilecek yerler ve sokak yemekleri turları hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Bangkok",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Bangkok'ta sokak yemekleri turları"}
            )
        ]
    ),
    Row(
        question="Berlin'de 3 günlük bir iş gezisi planlıyorum. Gezilecek yerler ve akşam aktiviteleri hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Berlin",
                    "limit": 5
                }
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Berlin'de iş gezisi için akşam aktiviteleri"}
            )
        ]
    ),
    Row(
        question="Dubai'de 5 günlük bir tatil planlıyorum. Gezilecek yerler ve çöl safarisi turları hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Dubai",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Dubai çöl safarisi turları"}
            )
        ]
    ),
    Row(
        question="Los Angeles'ta yaz tatili için gezilecek yerler ve plaj aktiviteleri hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Los Angeles",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Los Angeles'ta plaj aktiviteleri"}
            )
        ]
    ),
    Row(
        question="Cape Town'da 1 haftalık bir tatil planlıyorum. Gezilecek yerler ve safari turları hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Cape Town",
                    "limit": 10
                }
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Cape Town safari turları"}
            )
        ]
    ),
    Row(
        question="Melbourne'de 4 günlük bir iş gezisi planlıyorum. Gezilecek yerler ve şehirde yapılacak akşam aktiviteleri hakkında bilgi alabilir misin?",
        function_calls=[
            FunctionCall(
                name="list_attractions",
                arguments={
                    "city": "Melbourne",
                    "limit": 5
                }
            ),
            FunctionCall(
                name="search_web",
                arguments={"query": "Melbourne'de iş gezisi için akşam aktiviteleri"}
            )
        ]
    )
    
]

In [196]:
len(attraction_data)

30

In [197]:
list_attraction_function_calls = generate_function_call_data(attraction_data, functions)

In [198]:
save_json(list_attraction_function_calls, "list_attraction_function_calls.json")

### Search Youtube

In [ ]:
youtube_data = [

    Row(

        question="Paris'te gezilecek en popüler yerler nereler?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Paris gezilecek yerler vlog"}

            )

        ]

    ),

    Row(

        question="Roma'da 3 günlük bir gezi için öneriler nelerdir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Roma 3 gün gezi rehberi"}

            )

        ]

    ),

    Row(

        question="Bali'de uygun fiyatlı oteller ve gezilecek yerler hakkında bilgi alabilir miyim?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Bali uygun oteller ve gezilecek yerler"}

            )

        ]

    ),

    Row(

        question="Barselona'da gece hayatı nasıldır?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Barselona gece hayatı vlog"}

            )

        ]

    ),

    Row(

        question="Londra'da çocuklarla yapılacak aktiviteler nelerdir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Londra çocuklarla gezilecek yerler"}

            )

        ]

    ),

    Row(

        question="Kapadokya'da balon turu deneyimi nasıl?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Kapadokya balon turu deneyimi"}

            )

        ]

    ),

    Row(

        question="New York'ta alışveriş için en iyi bölgeler hangileri?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "New York alışveriş rehberi"}

            )

        ]

    ),

    Row(

        question="Tokyo'da geleneksel Japon yemekleri nerede yenir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Tokyo geleneksel Japon yemekleri"}

            )

        ]

    ),

    Row(

        question="Amsterdam'da bisikletle şehir turu nasıl yapılır?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Amsterdam bisiklet turu vlog"}

            )

        ]

    ),

    Row(

        question="Dubai'de çöl safarisi ve alışveriş merkezleri hakkında bilgi verir misin?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Dubai çöl safarisi deneyimi"}

            ),

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Dubai alışveriş merkezleri vlog"}

            )

        ]

    ),

              

               Row(

        question="Viyana'da 2 günlük bir kültür turu nasıl planlanır?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Viyana 2 gün kültür turu planı"}

            )

        ]

    ),

    Row(

        question="Prag'da hafta sonu gezisi için önerilen rotalar nelerdir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Prag hafta sonu gezi rotası"}

            )

        ]

    ),

    Row(

        question="Lizbon'da 5 günlük tatil için en iyi aktiviteler neler?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Lizbon 5 gün tatil aktiviteleri"}

            )

        ]

    ),

    Row(

        question="Atina'da tarihi yerleri gezmek için 3 günlük bir tur önerir misin?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Atina 3 gün tarihi tur"}

            )

        ]

    ),

    Row(

        question="Münih'te 1 günlük şehir turu nasıl yapılır?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Münih 1 gün şehir turu"}

            )

        ]

    ),

    Row(

        question="Venedik'te romantik bir hafta sonu için yapılacaklar listesi nedir?",

       function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Venedik romantik hafta sonu önerileri"}

            )

        ]

    ),

    Row(

        question="Madrid'de 4 günlük sanat ve gastronomi turu önerileri nelerdir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Madrid 4 gün sanat gastronomi turu"}

            )

        ]

    ),

    Row(

        question="Kopenhag'da bisikletle 2 günlük gezi rotası önerir misin?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Kopenhag bisiklet 2 gün gezi rotası"}

            )

        ]

    ),

    Row(

        question="Edinburgh'da 3 gün boyunca gezilecek tarihi ve doğal yerler nereler?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Edinburgh 3 gün tarihi doğal yerler"}

            )

        ]

    ),

    Row(

        question="Seul'de 1 haftalık turistik gezi için en iyi plan nedir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Seul 1 hafta turistik gezi planı"}

            )

        ]

    ),

              

               Row(

        question="Prag'da 2 günlük bir gezi için en iyi rota nedir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Prag 2 gün gezi vlog"}

            ),

            FunctionCall(

                name="search_web",

                arguments={"query": "Prag 2 günde gezilecek yerler ve öneriler"}

            )

        ]

    ),

    Row(

        question="Kopenhag'da vegan restoranlar ve popüler kafeler nerede bulunur?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Kopenhag vegan restoranlar ve kafeler"}

            ),

            FunctionCall(

                name="search_web",

                arguments={"query": "Kopenhag en iyi vegan restoranlar ve kafeler"}

            )

        ]

    ),

    Row(

        question="Viyana'da 3 günlük kültür turu için müze ve sanat galerisi önerileri nelerdir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Viyana 3 gün müze sanat galerisi turu"}

            ),

            FunctionCall(

                name="search_web",

                arguments={"query": "Viyana'da gezilecek müzeler ve sanat galerileri"}

            )

        ]

    ),

    Row(

        question="Lizbon'da gece hayatı ve barlar hakkında bilgi alabilir miyim?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Lizbon gece hayatı ve barlar vlog"}

            ),

            FunctionCall(

                name="search_web",

                arguments={"query": "Lizbon gece hayatı ve en iyi barlar"}

            )

        ]

    ),

    Row(

        question="Amsterdam'da 4 günlük bisiklet turu için rota ve konaklama önerileri nelerdir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Amsterdam 4 gün bisiklet turu vlog"}

            ),

            FunctionCall(

                name="search_web",

                arguments={"query": "Amsterdam bisiklet turu rotaları ve konaklama"}

            )

        ]

    ),

    Row(

        question="Atina'da tarihi mekanlar ve yerel yemekler nerede bulunur?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Atina tarihi mekanlar ve yerel yemekler"}

            ),

            FunctionCall(

                name="search_web",

                arguments={"query": "Atina'da gezilecek tarihi yerler ve yöresel yemekler"}

            )

        ]

    ),

    Row(

        question="Münih'te 2 günlük aile gezisi için çocuk dostu aktiviteler nelerdir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Münih çocuklarla gezilecek yerler vlog"}

            ),

            FunctionCall(

                name="search_web",

                arguments={"query": "Münih'te çocuklar için aktiviteler ve aile gezisi önerileri"}

            )

        ]

    ),

    Row(

        question="Edinburgh'da 3 gün boyunca doğa yürüyüşü yapılacak en iyi parklar hangileridir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Edinburgh doğa yürüyüşü parkları vlog"}

            ),

            FunctionCall(

                name="search_web",

                arguments={"query": "Edinburgh'da doğa yürüyüşü için en iyi parklar"}

            )

        ]

    ),

    Row(

        question="Tokyo'da alışveriş ve elektronik mağazaları nerede bulunur?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Tokyo alışveriş elektronik mağazalar vlog"}

            ),

            FunctionCall(

                name="search_web",

                arguments={"query": "Tokyo'da alışveriş ve elektronik mağazalar"}

            )

        ]

    ),

    Row(

        question="Seul'de 5 günlük gezi için en iyi alışveriş ve kültürel etkinlikler nelerdir?",

        function_calls=[

            FunctionCall(

                name="search_youtube",

                arguments={"query": "Seul 5 gün alışveriş ve kültürel etkinlikler"}

            ),

            FunctionCall(

                name="search_web",

                arguments={"query": "Seul'de alışveriş ve kültürel etkinlikler rehberi"}

            )

        ]

    ),

]

In [200]:
len(youtube_data)

30

In [201]:
youtube_search_data = generate_function_call_data(youtube_data, functions)

In [209]:
save_json(youtube_search_data, "youtube_search_function_calls.json")

### Event Data

In [205]:
event_data = [

    Row(

        question="Paris'te Eylül ayında ulaşımı etkileyebilecek etkinlikler veya grevler var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Paris", "startDate": "2025-09-01", "endDate": "2025-09-30"}

            )

        ]

    ),

    Row(

        question="Londra'da Temmuz ayında hangi resmi tatiller ve büyük etkinlikler var?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Londra", "startDate": "2025-07-01", "endDate": "2025-07-31"}

            )

        ]

    ),

    Row(

        question="Roma'da Mayıs ayında seyahati etkileyebilecek festivaller veya toplu taşıma grevleri var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Roma", "startDate": "2025-05-01", "endDate": "2025-05-31"}

            )

        ]

    ),

    Row(

        question="Berlin'de Ağustos ayında gerçekleşecek büyük konserler veya karnavallar nelerdir?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Berlin", "startDate": "2025-08-01", "endDate": "2025-08-31"}

            )

        ]

    ),

    Row(

        question="Amsterdam'da Nisan ayında ulaşımı zorlaştırabilecek etkinlikler var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Amsterdam", "startDate": "2025-04-01", "endDate": "2025-04-30"}

            )

        ]

    ),

    Row(

        question="Madrid'de Haziran ayında resmi tatiller veya büyük spor etkinlikleri var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Madrid", "startDate": "2025-06-01", "endDate": "2025-06-30"}

            )

        ]

    ),

    Row(

        question="Viyana'da Aralık ayında Noel pazarları dışında başka hangi etkinlikler var?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Viyana", "startDate": "2025-12-01", "endDate": "2025-12-31"}

            )

        ]

    ),

    Row(

        question="Prag'da Ocak ayında ulaşımı etkileyebilecek grevler veya gösteriler var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Prag", "startDate": "2025-01-01", "endDate": "2025-01-31"}

            )

        ]

    ),

    Row(

        question="Barcelona'da Ekim ayında hangi festivaller veya toplu taşıma grevleri olabilir?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Barcelona", "startDate": "2025-10-01", "endDate": "2025-10-31"}

            )

        ]

    ),

    Row(

        question="İstanbul'da Nisan ayında ulaşımı etkileyebilecek maraton veya büyük etkinlikler var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "İstanbul", "startDate": "2025-04-01", "endDate": "2025-04-30"}

            )

        ]

    ),

              

               Row(

        question="Londra ve Paris'te Ekim ayında ulaşımı etkileyebilecek etkinlikler veya resmi tatiller var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Londra", "startDate": "2025-10-01", "endDate": "2025-10-31"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Paris", "startDate": "2025-10-01", "endDate": "2025-10-31"}

            )

        ]

    ),

    Row(

        question="Berlin ve Amsterdam'da Haziran ayında gerçekleşecek büyük festivaller ve grevler nelerdir?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Berlin", "startDate": "2025-06-01", "endDate": "2025-06-30"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Amsterdam", "startDate": "2025-06-01", "endDate": "2025-06-30"}

            )

        ]

    ),

    Row(

        question="İstanbul ve Roma'da Mayıs ayında ulaşımı etkileyebilecek maraton, grev veya büyük etkinlikler var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "İstanbul", "startDate": "2025-05-01", "endDate": "2025-05-31"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Roma", "startDate": "2025-05-01", "endDate": "2025-05-31"}

            )

        ]

    ),

    Row(

        question="Madrid, Barselona ve Lizbon'da Temmuz ayında hangi festivaller ve resmi tatiller var?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Madrid", "startDate": "2025-07-01", "endDate": "2025-07-31"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Barselona", "startDate": "2025-07-01", "endDate": "2025-07-31"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Lizbon", "startDate": "2025-07-01", "endDate": "2025-07-31"}

            )

        ]

    ),

    Row(

        question="Viyana ve Prag'da Aralık ayında ulaşımı etkileyebilecek Noel etkinlikleri veya grevler var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Viyana", "startDate": "2025-12-01", "endDate": "2025-12-31"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Prag", "startDate": "2025-12-01", "endDate": "2025-12-31"}

            )

        ]

    ),

   Row(

        question="Tokyo ve Seul'de Ağustos ayında gerçekleşecek büyük festivaller, resmi tatiller veya ulaşımı etkileyebilecek etkinlikler nelerdir?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Tokyo", "startDate": "2025-08-01", "endDate": "2025-08-31"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Seul", "startDate": "2025-08-01", "endDate": "2025-08-31"}

            )

        ]

    ),

    Row(

        question="Paris, Londra ve Berlin'de Mart ayında ulaşımı etkileyebilecek grevler veya gösteriler var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Paris", "startDate": "2025-03-01", "endDate": "2025-03-31"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Londra", "startDate": "2025-03-01", "endDate": "2025-03-31"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Berlin", "startDate": "2025-03-01", "endDate": "2025-03-31"}

            )

        ]

    ),

    Row(

        question="Barselona ve Madrid'de Eylül ayında hangi spor etkinlikleri ve festivaller var?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Barselona", "startDate": "2025-09-01", "endDate": "2025-09-30"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Madrid", "startDate": "2025-09-01", "endDate": "2025-09-30"}

            )

        ]

    ),

    Row(

        question="Amsterdam ve Brüksel'de Nisan ayında ulaşımı etkileyebilecek etkinlikler veya resmi tatiller var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Amsterdam", "startDate": "2025-04-01", "endDate": "2025-04-30"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Brüksel", "startDate": "2025-04-01", "endDate": "2025-04-30"}

            )

        ]

    ),

    Row(

        question="Kopenhag ve Stockholm'de Haziran ayında gerçekleşecek büyük konserler, festivaller veya resmi tatiller nelerdir?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Kopenhag", "startDate": "2025-06-01", "endDate": "2025-06-30"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Stockholm", "startDate": "2025-06-01", "endDate": "2025-06-30"}

            )

        ]

    ),

              

               Row(

        question="İstanbul'da 2025 genel seçimleri sırasında ulaşımı etkileyebilecek etkinlikler veya kısıtlamalar var mı?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "İstanbul", "startDate": "2025-06-01", "endDate": "2025-06-10"}

            )

        ]

    ),

    Row(

        question="Paris'te Paskalya döneminde hangi dini etkinlikler ve resmi tatiller var?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Paris", "startDate": "2025-04-18", "endDate": "2025-04-22"}

            )

        ]

    ),

    Row(

        question="Madrid'de Real Madrid ve Barcelona arasında oynanacak büyük futbol maçları ve bu tarihlerdeki etkinlikler nelerdir?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Madrid", "startDate": "2025-03-01", "endDate": "2025-03-31"}

            )

        ]

    ),

    Row(

        question="Berlin'de Almanya Ulusal Günü'nde (3 Ekim) hangi kutlamalar ve toplu taşıma değişiklikleri olabilir?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Berlin", "startDate": "2025-10-03", "endDate": "2025-10-03"}

            )

        ]

    ),

    Row(

        question="Londra'da yılbaşı gecesi ve ertesi gün hangi etkinlikler ve ulaşım kısıtlamaları var?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Londra", "startDate": "2025-12-31", "endDate": "2026-01-01"}

            )

        ]

    ),

    Row(

        question="Roma'da Paskalya ve Ferragosto dönemlerinde hangi dini ve ulusal etkinlikler var?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Roma", "startDate": "2025-04-18", "endDate": "2025-04-22"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Roma", "startDate": "2025-08-15", "endDate": "2025-08-15"}

            )

        ]

    ),

    Row(

        question="Amsterdam'da Kraliçe Günü ve büyük futbol maçları sırasında şehirde hangi etkinlikler var?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Amsterdam", "startDate": "2025-04-27", "endDate": "2025-04-27"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Amsterdam", "startDate": "2025-05-10", "endDate": "2025-05-12"}

            )

        ]

    ),

    Row(

        question="Atina'da Ortodoks Paskalyası ve ulusal bayramlarda hangi etkinlikler ve kısıtlamalar var?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Atina", "startDate": "2025-04-18", "endDate": "2025-04-21"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Atina", "startDate": "2025-03-25", "endDate": "2025-03-25"}

            )

        ]

    ),

    Row(

        question="Londra'da Wimbledon tenis turnuvası ve aynı dönemdeki diğer büyük etkinlikler nelerdir?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Londra", "startDate": "2025-06-30", "endDate": "2025-07-14"}

            )

        ]

    ),

    Row(

        question="Münih'te Oktoberfest ve Almanya milli maçları sırasında şehirde hangi etkinlikler var?",

        function_calls=[

            FunctionCall(

                name="list_events",

                arguments={"city": "Münih", "startDate": "2025-09-20", "endDate": "2025-10-05"}

            ),

            FunctionCall(

                name="list_events",

                arguments={"city": "Münih", "startDate": "2025-10-10", "endDate": "2025-10-15"}

            )

        ]

    ),

]

In [206]:
len(event_data)

30

In [207]:
event_function_calls = generate_function_call_data(event_data, functions)

In [208]:
save_json(event_function_calls, "list_event_function_call_data.json")

## Combine Data

In [219]:
import json

def load_json(filename):
    with open(filename, 'r', encoding="utf-8") as f:
        return json.load(f)

In [220]:
flight_calls = load_json("flight_search_function_calls.json")
attraction_calls = load_json("list_attraction_function_calls.json")
youtube_calls = load_json("youtube_search_function_calls.json")
event_calls = load_json("list_event_function_call_data.json")
web_search_calls = load_json("web_search_function_calls.json")
hotel_calls = load_json("hotel_search_function_calls.json")

all_function_calls = flight_calls + attraction_calls + youtube_calls + event_calls + web_search_calls + hotel_calls

In [221]:
len(all_function_calls)

236

In [268]:
all_function_calls[0]['function_calls'][0]

{'name': 'list_attractions', 'arguments': {'city': 'Tokyo', 'limit': 7}}

In [267]:
all_function_calls[0]['available_functions'][0]

{'name': 'list_attractions',
 'description': 'List top attractions for the given city. Returns an array of attraction objects (name, address, shortDescription, rating, category).',
 'parameters': {'type': 'object',
  'properties': {'city': {'type': 'string',
    'description': "City name or query (e.g., 'Paris, France' or 'Seattle, WA')."},
   'limit': {'type': 'integer',
    'description': 'Maximum number of attractions to return.',
    'minimum': 1,
    'default': 10}},
  'required': ['city']}}

## Generate Dataset

In [ ]:
import os
os.environ["HF_TOKEN"]="xxxxxx"

In [262]:
from datasets import Dataset
import random
import json

random.shuffle(all_function_calls)

question = [call["question"] for call in all_function_calls]
function_calls = [json.dumps(call["function_calls"], ensure_ascii=False) for call in all_function_calls]
available_functions = [ json.dumps(call["available_functions"], ensure_ascii=False) for call in all_function_calls ]

function_call_dataset = Dataset.from_dict({"question": question, "function_calls": function_calls, "available_functions": available_functions})

In [265]:
function_call_dataset.push_to_hub("habanoz/travel_function_call_dataset", private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading files as a binary IO buffer is not supported by Xet Storage. Falling back to HTTP upload.


README.md:   0%|          | 0.00/370 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/habanoz/travel_function_call_dataset/commit/4c11b57f8c9f90be2b7dac8f2bc7010d6ddccaa2', commit_message='Upload dataset', commit_description='', oid='4c11b57f8c9f90be2b7dac8f2bc7010d6ddccaa2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/habanoz/travel_function_call_dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='habanoz/travel_function_call_dataset'), pr_revision=None, pr_num=None)